In [17]:
import os
import re
from pathlib import Path
from collections import Counter

import yaml

In [8]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%autoreload 2

In [10]:
import mushroom.data.multiplex as multiplex

In [11]:
out_dir = '/data/estorrs/mushroom/data/examples/HT397B1_v1/mushroom_outputs'
Path(out_dir).mkdir(parents=True, exist_ok=True)

In [12]:
config = yaml.safe_load(
    open('/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/mushroom_inputs.yaml')
)
config

[{'data': [{'dtype': 'he',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s0_he.tif'},
   {'dtype': 'visium',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s0_visium.h5ad'}],
  'id': 's0',
  'position': 0},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s1_multiplex.ome.tiff'}],
  'id': 's1',
  'position': 1},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s2_multiplex.ome.tiff'}],
  'id': 's2',
  'position': 11},
 {'data': [{'dtype': 'he',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s3_he.tif'},
   {'dtype': 'visium',
    'filepath': '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s3_visium.h5ad'}],
  'id': 's3',
  'position': 20},
 {'data': [{'dtype': 'multiplex',
    'filepath': '/

## train multiplex

In [22]:
# only relavent for dinglab data since our channel names are all over the place
# will remove for official example
official_to_options = {
    'Pan-Cytokeratin': ['Pan-Cytokeratin', 'Pan-CK', 'Pan-CK (D)', 'PanCK (D)'],
    'E-cadherin': ['E-cadherin'],
    'CD45': ['CD45 (D)', 'CD45', 'CD45-(D)'],
    'CD45RO': ['CD45RO'],
    'CD45RA': ['CD45RA'],
    'CD8': ['CD8', 'CD8a'],
    'DAPI': ['DAPI'],
    'CD4': ['CD4', 'CD4 (D)'],
    'CD3e': ['CD3e', 'CD3'],
    'Vimentin': ['Vimentin-(D)', 'Vimentin', 'Vimentin (D)'],
    'SMA': ['SMA-(D)', 'SMA', 'SMA (D)', 'a-SMA (D)'],
    'CD31': ['CD31', 'CD31-(D)', 'CD31 (D)'],
    'CD20': ['CD20-(D)', 'CD20', 'CD20 (D)', 'CD20-Akoya'],
    'CD68': ['CD68', 'CD68 (D)', 'CD68-(D)'],
    'CD163': ['CD163'],
    'FOXP3': ['FoxP3', 'FOXP3', 'Foxp3'],
    'cKit': ['cKIT (D)', 'cKit (D)', 'cKit', 'cKIT', 'cKIT-(D)'],
    'MGP': ['MGP', 'MGP (D)', 'MGP-(D)'],
    'CD36': ['CD36-(D)', 'CD36'],
    'PR': ['PR', 'PR-(D)', 'PR (D)'],
    'ER': ['ER', 'ER-(D)', 'ER (D)'],
    'P21': ['P21', 'P21-(D)', 'P21-(D)', 'P21 (D)'],
    'P16': ['P16-(D)'],
    'CK5': ['Keratin 5', 'KRT5'],
    'TFF1': ['TFF1', 'TFF1-(D)', 'TFF1 (D)'],
    'beta-integrin': ['beta-integrin', 'beta3-integrin'],
    'CK14': ['CK14', 'Keratin 14', 'CK14 (D)'],
    'CK17': ['CK17', 'CK17 (D)', 'CK17-(D)', 'Keratin 17'],
    'CK19': ['CK19', 'CK19 (D)', 'CK19-(D)'],
    'CD11b': ['CD11b', 'CD11b (D)', 'CD11b-(D)'],
    'GATA3': ['GATA3', 'GATA3-(D)', 'GATA3 (D)'],
    'PLAT/tPA': ['PLAT/tPA', 'PLAT/tPA (D)'],
    'COX6c': ['COX6c (D)'],
    'Her2': ['Her2', 'Her2 (D)'],
    'Bap1': ['Bap1 (D)'],
    'GLUT1': ['Glut1', 'GLUT1', 'GLUT1-(D)', 'GLUT1 (D)'],
    'CD11c': ['CD11c'],
    'HLA-DR': ['HLA-DR'],
    'Ki67': ['Ki67', 'KI67'],
    'Podoplanin': ['Podoplanin', 'PDPN', 'Podoplanin (D)'],
    'CTLA4': ['CTLA4', 'CTLA4 (D)', 'CTLA4-(D)'],
    'SLC39A6': ['SLC39A6'],
    'BCA1': ['BCA1'],
    'BCAL': ['BCAL'],
    'TUBB3': ['TUBB3', 'TUBB3 (D)'],
    'PTPRZ1': ['PTPRZ1', 'PTPRZ1 (D)'],
    'HIF1A': ['HIF1a', 'HIF1a (D)', 'HIF1A'],
    'PAI1': ['PAI1', 'PAI1-(D)'],
    'GFAP': ['GFAP', 'GFAP (D)'],
    'VEGFA': ['VEGFA', 'VEGFA (D)'],
    'IBA1': ['IBA1', 'IBA1 (D)'],
    'OLIG2': ['OLIG2', 'OLIG2 (D)'],
    'FN1': ['FN1', 'FN1 (D)'],
    'a-Amylase': ['a-Amylase'],
    'Hep-Par-1': ['Hep-Par-1 (D)'],
    'Granzyme-B': ['Granzyme B'],
    'TCF-1': ['TCF-1'],
    'CD39': ['CD39'],
    'PD1': ['PD-1'],
    'PDL1': ['PD-L1'],
    'Histone-H3-Pho': ['Histone H3 Pho'],
    'Maspin': ['Maspin'],
    'MMP9': ['MMP9'],
    'CD44': ['CD44'],
    'CD107A': ['CD107a'],
    'FGFR3': ['FGFR3'],
    'CD138': ['CD138'],
    'MLPH': ['MLPH'],
    'P63': ['P63'],
    'GP2': ['GP2'],
    'COX2': ['COX2'],
    'Lyve-1': ['Lyve-1'],
}
channel_mapping = {v:k for k, vs in official_to_options.items() for v in vs}

In [15]:
fps = [d['filepath'] for entry in config for d in entry['data'] if d['dtype']=='multiplex']
fps

['/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s1_multiplex.ome.tiff',
 '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s2_multiplex.ome.tiff',
 '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s4_multiplex.ome.tiff',
 '/data/estorrs/mushroom/data/examples/HT397B1_v1/registration/registered/s5_multiplex.ome.tiff']

In [23]:
multiplex.get_common_channels(fps, channel_mapping)

['Bap1',
 'CD31',
 'CD45',
 'CD68',
 'CD8',
 'CK14',
 'CK19',
 'COX6c',
 'DAPI',
 'ER',
 'FOXP3',
 'GATA3',
 'GLUT1',
 'HLA-DR',
 'Her2',
 'Ki67',
 'MGP',
 'PLAT/tPA',
 'PR',
 'Pan-Cytokeratin',
 'Podoplanin',
 'SMA',
 'Vimentin',
 'cKit']

In [19]:
channels = sorted([c for c, count in Counter(pool).items() if count==len(fps)])
channels

['Bap1',
 'CD31',
 'CD45',
 'CD68',
 'CD8',
 'CK14',
 'CK19',
 'COX6c',
 'DAPI',
 'ER',
 'FOXP3',
 'GATA3',
 'GLUT1',
 'HLA-DR',
 'Her2',
 'Ki67',
 'MGP',
 'PLAT/tPA',
 'PR',
 'Pan-Cytokeratin',
 'Podoplanin',
 'SMA',
 'Vimentin',
 'cKit']

In [ ]:
scale = .1
slide_to_img = {}
for fp in fps:
    sample = fp.split('/')[-1].replace('.ome.tiff', '')
    cs, img = utils.extract_ome_tiff(fp, as_dict=False)
    img = torch.tensor(img)
    thumbnail = TF.resize(img, (int(scale * img.shape[-2]), int(scale * img.shape[-1])))
    thumbnail = thumbnail.to(torch.float32)
    
    cs = [utils.R_CHANNEL_MAPPING[c] for c in cs]
    idxs = [cs.index(c) for c in channels]
    thumbnail = thumbnail[idxs]
    
    slide_to_img[sample] = thumbnail

In [24]:
from dataclasses import dataclass
@dataclass
class VitArgs:
    image_size: int = 256
    patch_size: int = 32
    dim: int = 1024
    depth: int = 6
    heads: int = 8
    mlp_dim: int = 2048
    channels: int = 3

In [25]:
args = VitArgs()

In [29]:
import torch

In [30]:
torch.device

torch.device

In [32]:
torch.cuda.is_available()

True

In [36]:
torch.cuda.device_count()

4

In [41]:
x = torch.rand(1, 1, 1, device='cuda')

In [42]:
x.device

device(type='cuda', index=0)

In [67]:
class MyModel(torch.nn.Module):
    def __init__(self, a='hello'):
        super().__init__()
        self.m = torch.nn.Sequential(
            torch.nn.Linear(10, 10),
        )
        self.a = a

    def forward(self, x):
        return self.m(x)

In [68]:
m = MyModel(a='world')

In [69]:
torch.save(m.state_dict(), '/data/estorrs/sandbox/test.pt')

In [49]:
import pickle

In [52]:
with open('/data/estorrs/sandbox/test.pt', 'wb') as handle:
    pickle.dump(m, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
with open('/data/estorrs/sandbox/test.pt', 'rb') as handle:
    b = pickle.load(handle)

In [70]:
b = torch.load('/data/estorrs/sandbox/test.pt')
b

OrderedDict([('m.0.weight',
              tensor([[ 0.2271, -0.0835,  0.1331,  0.0984, -0.1592, -0.0435,  0.1436,  0.2511,
                        0.0489, -0.0235],
                      [ 0.2308,  0.0871, -0.1251, -0.2266, -0.1047, -0.2408,  0.0744, -0.1246,
                       -0.3153, -0.2009],
                      [ 0.2196, -0.1324,  0.1127,  0.0433,  0.0710,  0.2446, -0.1160,  0.0699,
                       -0.0647, -0.0979],
                      [-0.0448,  0.3034, -0.0991, -0.2263, -0.1555,  0.2916, -0.0454,  0.2337,
                        0.0383, -0.0366],
                      [-0.2109,  0.1334, -0.0319, -0.1295,  0.2019, -0.2051,  0.0440, -0.1576,
                        0.2542,  0.2031],
                      [ 0.1629, -0.2477,  0.0604, -0.2224, -0.2191,  0.1731, -0.1431,  0.0859,
                       -0.0729, -0.2662],
                      [ 0.2087, -0.1349,  0.2903, -0.0050,  0.2802,  0.2694,  0.2463,  0.1801,
                        0.2321,  0.1710],
             

In [71]:
b.a

AttributeError: 'collections.OrderedDict' object has no attribute 'a'